https://theconversation.com/momentum-isnt-magic-vindicating-the-hot-hand-with-the-mathematics-of-streaks-74786

In [1]:
using Distributions
using Random

function prob(X, k)
    n = length(X)
    den = 0
    num = 0
    for i in k+1:n
        if all(X[i-j] for j in 1:k)
            den += 1
            num += X[i]
        end
    end
    num/den
end

function sim(n, k; L=10^7)
    nths = Threads.nthreads()
    Xtmp = [trues(n) for _ in 1:nths]
    P = zeros(nths)
    N = zeros(Int, nths)
    Threads.@threads for _ in 1:L
        tid = Threads.threadid()
        X = rand!(Xtmp[tid])
        p = prob(X, k)
        if !isnan(p)
            N[tid] += 1
            P[tid] += p
        end
    end
    sum(N)/L, sum(P)/sum(N)
end

function sim1(n, k; L=10^7)
    nths = Threads.nthreads()
    Xtmp = [trues(n) for _ in 1:nths]
    M = zeros(Int, nths)
    N = zeros(Int, nths)
    Threads.@threads for _ in 1:L
        tid = Threads.threadid()
        X = rand!(Xtmp[tid])
        i = rand(k+1:n)
        if all(X[i-j] for j in 1:k)
            N[tid] += 1
            M[tid] += X[i]
        end
    end
    sum(N)/L, sum(M)/sum(N)
end

function isadmissible(X, k)
    c = 0
    for x in X[begin:end-1]
        if x
            c += 1
            c ≥ k && return true
        else
            c = 0
        end
    end
    false
end

function sim2(n, k; L=10^7)
    nths = Threads.nthreads()
    Xtmp = [trues(n) for _ in 1:nths]
    M = zeros(Int, nths)
    N = zeros(Int, nths)
    Threads.@threads for _ in 1:L
        tid = Threads.threadid()
        X = rand!(Xtmp[tid])
        isadmissible(X, k) || continue
        N[tid] += 1
        c = 0
        while true
            c += 1
            i = rand(k+1:n)
            if all(X[i-j] for j in 1:k)
                M[tid] += X[i]
                break
            end
        end
    end
    sum(N)/L, sum(M)/sum(N)
end

function alladmissibles(n, k)
    @assert n ≤ 20
    A = Tuple{NTuple{n, Bool}, Vector{Bool}}[]
    for X in Iterators.product(fill((false, true), n)...)
        if isadmissible(X, k)
            Y = Bool[]
            for i in 1+k:n
                if all(X[i-j] for j in 1:k)
                    push!(Y, X[i])
                end
            end
            push!(A, (X, Y))
        end
    end
    A
end

alladmissibles (generic function with 1 method)

In [2]:
alladmissibles(5, 3)

6-element Vector{Tuple{NTuple{5, Bool}, Vector{Bool}}}:
 ((1, 1, 1, 0, 0), [0])
 ((0, 1, 1, 1, 0), [0])
 ((1, 1, 1, 1, 0), [1, 0])
 ((1, 1, 1, 0, 1), [0])
 ((0, 1, 1, 1, 1), [1])
 ((1, 1, 1, 1, 1), [1, 1])

In [3]:
alladmissibles(5, 3) .|> (t -> t[2]) .|> mean |> mean

0.4166666666666667

In [4]:
@show sim(5, 3) sim1(5, 3) sim2(5, 3);

sim(5, 3) = (0.1876402, 0.4163582750391441)
sim1(5, 3) = (0.1248436, 0.4996211259527921)
sim2(5, 3) = (0.1873954, 0.4160331576975742)


In [5]:
alladmissibles(10, 3)

476-element Vector{Tuple{NTuple{10, Bool}, Vector{Bool}}}:
 ((1, 1, 1, 0, 0, 0, 0, 0, 0, 0), [0])
 ((0, 1, 1, 1, 0, 0, 0, 0, 0, 0), [0])
 ((1, 1, 1, 1, 0, 0, 0, 0, 0, 0), [1, 0])
 ((1, 1, 1, 0, 1, 0, 0, 0, 0, 0), [0])
 ((0, 0, 1, 1, 1, 0, 0, 0, 0, 0), [0])
 ((1, 0, 1, 1, 1, 0, 0, 0, 0, 0), [0])
 ((0, 1, 1, 1, 1, 0, 0, 0, 0, 0), [1, 0])
 ((1, 1, 1, 1, 1, 0, 0, 0, 0, 0), [1, 1, 0])
 ((1, 1, 1, 0, 0, 1, 0, 0, 0, 0), [0])
 ((0, 1, 1, 1, 0, 1, 0, 0, 0, 0), [0])
 ((1, 1, 1, 1, 0, 1, 0, 0, 0, 0), [1, 0])
 ((1, 1, 1, 0, 1, 1, 0, 0, 0, 0), [0])
 ((0, 0, 0, 1, 1, 1, 0, 0, 0, 0), [0])
 ⋮
 ((0, 0, 1, 0, 1, 1, 1, 1, 1, 1), [1, 1, 1])
 ((1, 0, 1, 0, 1, 1, 1, 1, 1, 1), [1, 1, 1])
 ((0, 1, 1, 0, 1, 1, 1, 1, 1, 1), [1, 1, 1])
 ((1, 1, 1, 0, 1, 1, 1, 1, 1, 1), [0, 1, 1, 1])
 ((0, 0, 0, 1, 1, 1, 1, 1, 1, 1), [1, 1, 1, 1])
 ((1, 0, 0, 1, 1, 1, 1, 1, 1, 1), [1, 1, 1, 1])
 ((0, 1, 0, 1, 1, 1, 1, 1, 1, 1), [1, 1, 1, 1])
 ((1, 1, 0, 1, 1, 1, 1, 1, 1, 1), [1, 1, 1, 1])
 ((0, 0, 1, 1, 1, 1, 1, 1, 1, 1), [1, 1, 

In [6]:
alladmissibles(10, 3) .|> (t -> t[2]) .|> mean |> mean

0.35260604241696675

In [7]:
@show sim(10, 3) sim1(10, 3) sim2(10, 3);

sim(10, 3) = (0.4646717, 0.3525811864285721)
sim1(10, 3) = (0.12501, 0.5002375809935206)
sim2(10, 3) = (0.4649015, 0.35245229365790387)


In [8]:
@show sim(30, 3) sim1(30, 3) sim2(30, 3);

sim(30, 3) = (0.8997, 0.3823245268627625)
sim1(30, 3) = (0.1249244, 0.49969501554540185)
sim2(30, 3) = (0.899836, 0.38202105717041773)


In [9]:
@show sim(100, 3) sim1(100, 3) sim2(100, 3);

sim(100, 3) = (0.9997243, 0.46028864758069166)
sim1(100, 3) = (0.124846, 0.5000680838793393)
sim2(100, 3) = (0.9997115, 0.4602426800131838)


In [10]:
@show sim(300, 3) sim1(300, 3) sim2(300, 3);

sim(300, 3) = (1.0, 0.4878309976827274)
sim1(300, 3) = (0.1248072, 0.49957935119127744)
sim2(300, 3) = (1.0, 0.4879832)


In [11]:
@show sim(1000, 3) sim1(1000, 3) sim2(1000, 3);

sim(1000, 3) = (1.0, 0.49646528141497764)
sim1(1000, 3) = (0.1250088, 0.5001775874978401)
sim2(1000, 3) = (1.0, 0.4965632)
